In [1]:
import pandas as pd
import os
from functools import reduce
import unidecode

In [3]:
caminho_da_base = r"Processamento_base_SSP-SP\regioes(2010, a 2024)"
anos = os.listdir(caminho_da_base)

def dict_crimes(ano):
    dicionario = {}
    caminho = os.path.join(caminho_da_base, str(ano))
    
    for arquivo in os.listdir(caminho):
        if arquivo.endswith('.csv'):
            chave = arquivo.replace('.csv', '')
            caminho_completo = os.path.join(caminho, arquivo)
            dicionario[chave] = caminho_completo
    
    return dicionario

def criar_dataframe(ano):
    
    dataframes = []
    
    dicionario_caminhos = dict_crimes(ano)
    for tipo_crime, caminho_crime in dicionario_caminhos.items():
        df = pd.read_csv(caminho_crime)
        df = df[['cod_ibge', 'municipios', 'Total']]
        df.rename(columns={'Total': f'{tipo_crime.replace("_", " ").capitalize()} {ano}'}, inplace=True)
        dataframes.append(df)
    
    colunas_comuns = ['municipios', 'cod_ibge']

    df_final = reduce(lambda left, right: pd.merge(left, right, on=colunas_comuns, how='outer'), dataframes) 
    df_final.rename(columns={f'Total de estupro 4 {ano}': f'Total de estupro {ano}',
                             f'Total de roubo outros 1 {ano}': f'Total de Roubo {ano}',
                             f'Nº de vítimas em homicídio doloso 3 {ano}': f'Vítimas de homicídio {ano}',
                             "municipios": "Municipio"}, inplace=True)
    return df_final

In [ ]:
ocorrencias_2014 = criar_dataframe(2014)
ocorrencias_2016 = criar_dataframe(2016)
ocorrencias_2018 = criar_dataframe(2018) 
ocorrencias_2020 = criar_dataframe(2020)
ocorrencias_2022 = criar_dataframe(2022)

In [ ]:
caminho_est_pop = "Processamento_base_EST_POP\EST_POP_processada.csv"
est_pop = pd.read_csv(caminho_est_pop)

def merge_dfs(est_pop, df_ocorrencias, ano):
    
    est_pop = est_pop.loc[est_pop['Ano'] == ano].copy()
    pop = est_pop[['Ano', 'cod_ibge', 'Municipio', 'populacao']].copy()
    
    df_ocorrencias['Municipio'] = df_ocorrencias['Municipio'].apply(lambda x: unidecode.unidecode(x))
    pop['Municipio'] = pop['Municipio'].apply(lambda x: unidecode.unidecode(x))
    
    merged_df = pop.merge(df_ocorrencias, on=['cod_ibge', 'Municipio'], how='inner')
    merged_df.rename(columns={'populacao': f'Populacao {ano}'}, inplace=True)
    
    return merged_df
    
ocorrencias_2014 = merge_dfs(est_pop, ocorrencias_2014, 2014)
ocorrencias_2016 = merge_dfs(est_pop, ocorrencias_2016, 2016) 
ocorrencias_2018 = merge_dfs(est_pop, ocorrencias_2018, 2018) 
ocorrencias_2020 = merge_dfs(est_pop, ocorrencias_2020, 2020) 
ocorrencias_2022 = merge_dfs(est_pop, ocorrencias_2022, 2022) 

ocorrencias = [ocorrencias_2014, ocorrencias_2016,
               ocorrencias_2018, ocorrencias_2020,
               ocorrencias_2022]

In [5]:
pesos = {
    'Furto de veículo': 1,
    'Furto outros': 1,
    'Homicídio culposo outros': 4,
    'Homicídio culposo por acidente de trânsito': 4.5,
    'Latrocínio': 6,
    'Lesão corporal culposa outras': 2.5,
    'Lesão corporal culposa por acidente de trâsito': 3,
    'Lesão corporal dolosa': 3.5,
    'Vitimas homicidio': 5,
    'Tentativa de homicidio': 4.5,
    'Total estupro': 5,
    'Total roubos': 2
}

def calcular_indice_criminalidade(row, ano):
    numerador = sum((row[f"{crime} {ano}"] / row[f'Populacao {ano}']) * peso
                    for crime, peso in pesos.items() if f"{crime} {ano}" in row)
    denominador = sum(peso for crime, peso in pesos.items() if f"{crime} {ano}" in row)
    return numerador / denominador if denominador > 0 else 0

def normalizar_serie(serie):
    min_value = serie.min()
    max_value = serie.max()
    return (serie - min_value) / (max_value - min_value) if (max_value - min_value) > 0 else serie


In [ ]:
anos = ['2014', '2016', '2018', '2020','2022']

bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = ['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']
for df, ano in zip(ocorrencias, anos):
    df[f'Indice Criminalidade {ano}'] = df.apply(lambda row: calcular_indice_criminalidade(row, ano),
                                                 axis=1)
    df[f'Indice Criminalidade {ano}'] = normalizar_serie(df[f'Indice Criminalidade {ano}'])
    df[f'Classificação {ano}'] = pd.cut(df[f'Indice Criminalidade {ano}'],
                                        bins=bins,
                                        labels=labels,
                                        include_lowest=True)
    df.drop(columns=f'Populacao {ano}', inplace=True)

In [8]:
def exportar_df(df, ano, caminho_destino='indices_criminais'):
    diretorio = os.path.join(caminho_destino, str(ano))
    os.makedirs(diretorio, exist_ok=True)
    
    arquivo = os.path.join(diretorio, f"indices_criminais{ano}.csv")
    df.to_csv(arquivo, index=False)
    print(f"Exportado: {arquivo}")

In [9]:
exportar_df(ocorrencias_2014, 2014)
exportar_df(ocorrencias_2016, 2016)
exportar_df(ocorrencias_2018, 2018)
exportar_df(ocorrencias_2020, 2020)
exportar_df(ocorrencias_2022, 2022)

Exportado: indices_criminais\2014\indices_criminais2014.csv
Exportado: indices_criminais\2016\indices_criminais2016.csv
Exportado: indices_criminais\2018\indices_criminais2018.csv
Exportado: indices_criminais\2020\indices_criminais2020.csv
Exportado: indices_criminais\2022\indices_criminais2022.csv
